# Generators
***
A generator in python is a function that returns a set of items one at a time. In other words, you can iterate over the functions and retrieve each object.

Generators look exactly like functions but instead of returning elements, you `yield` elements.

In [37]:
def range_gen(n=3):
    i = 0
    while i < n:
        yield i
        i += 1

In [44]:
range_5 = range_gen(5)
for i in range_5:
    print(i, end=' ')
    
print()
# or just
for i in range_gen(5):
    print(i, end=' ')

0 1 2 3 4 
0 1 2 3 4 

Above we defined our first generator, `range_gen`. All it does, is feed us the numbers 0 to n, one at a time. 

Every time we call range_5 in the for loop, the generator **yields** the next value. This tells us that it maintains some sort of state remembering where it is. But why do we care? Why not just return the entire list of numbers from 0 to n and iterate over that in the for loop?

The reason is that generators are very memory efficient. Instead of creating and returning a potentially massive list of numbers (which can take up a lot of memory), the generator only has to store and give 1 number at a time.

In [48]:
import sys

In [51]:
def range_list(n):
    return list(range(n))

We'll compare the amount of bytes returned from the `range_list` function vs. the `range_gen` generator.

In [65]:
print(f'{sys.getsizeof(range_list(1_000_000)):,} bytes')
print(f'{sys.getsizeof(range_gen(1_000_000)):,} bytes')

9,000,120 bytes
128 bytes


We can see that the function takes up wayyyyy more memory than the generator does.

We don't just have to iterate over generators in a for loop. We can also call the `next` method on it to retrieve each value. But if we call `next` more times than there are values to yield than python will raise a StopIteration exception.

In [73]:
range_5 = range_gen(5)

print(next(range_5), end=' ')
print(next(range_5), end=' ')
print(next(range_5), end=' ')
print(next(range_5), end=' ')
print(next(range_5), end=' ')
# will raise a StopIteration exception
print(next(range_5))

0 1 2 3 4 

StopIteration: 